Inital EDA and Pre-processing

Steps left to do: hande missing values

In [69]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Read json sample into a dataframe
df = pd.read_json('/content/drive/MyDrive/MSBA Year 1 Sem 2/820 Unsupervised Machine Learning/820 Project Materials/random_sample_100.json')
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,vote
0,5,False,"01 23, 2017",A1YTZAZZ1L32RL,0002005263,{'Format:': ' Kindle Edition'},Chuck,Ok reading,Ok,1485129600,NaN,NaN
1,5,True,"08 3, 2014",A3EHZMD24B5BQ0,0002005263,{'Format:': ' Paperback'},Sola49,Good,Five Stars,1407024000,NaN,NaN
2,5,True,"07 9, 2013",A38ZMH9LRCZL3X,0001381733,{'Format:': ' Hardcover'},A. E. Stoff,I purchased it for my new great-granddaughter....,New Book for New Grandchild,1373328000,NaN,NaN
3,5,True,"12 3, 2017",AYMEYAL8G7LJB,0001384198,{'Format:': ' Hardcover'},MARILYN,A very nice replica of the original classic.,Excellent reproduction of the timeless classic.,1512259200,NaN,NaN
4,3,False,"10 4, 2011",A39QEKNEFSJ34Q,0002005263,{'Format:': ' Unknown Binding'},hrladyship,If you're thinking of buying The Sinister Pig ...,And the hero is . . .,1317686400,NaN,NaN


In [27]:
#Read csv with metadata by asin code
meta_data = pd.read_csv('/content/drive/MyDrive/MSBA Year 1 Sem 2/820 Unsupervised Machine Learning/820 Project Materials/books_meta_short.csv')

In [6]:
#Perform a left join to add metadata information (title, price) to each book for
#each book review
merged_df = pd.merge(df, meta_data, on='asin', how='left')

In [7]:
#Remove original dataframes
import gc #garbage collector for deleted dataframes

del df
del meta_data

gc.collect

<function gc.collect(generation=2)>

In [8]:
#Check dtypes and columns
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61100 entries, 0 to 61099
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   overall         61100 non-null  int64 
 1   verified        61100 non-null  bool  
 2   reviewTime      61100 non-null  object
 3   reviewerID      61100 non-null  object
 4   asin            61100 non-null  object
 5   style           58295 non-null  object
 6   reviewerName    61098 non-null  object
 7   reviewText      61094 non-null  object
 8   summary         61091 non-null  object
 9   unixReviewTime  61100 non-null  int64 
 10  image           109 non-null    object
 11  vote            11773 non-null  object
 12  Unnamed: 0      61100 non-null  int64 
 13  title           61100 non-null  object
 14  price           52443 non-null  object
dtypes: bool(1), int64(3), object(11)
memory usage: 7.1+ MB


In [9]:
#Convert style from dictionary into standard dataframe column
merged_df['style'] = merged_df['style'].apply(lambda x: x['Format:'].strip()\
                                if pd.notnull(x) and 'Format:' in x else x)

merged_df['style'].head()

0     Kindle Edition
1          Paperback
2          Hardcover
3          Hardcover
4    Unknown Binding
Name: style, dtype: object

In [10]:
#Drop image column since it is beyond of the scope of this project
merged_df.drop(columns='image', inplace=True)

In [29]:
#Dropping 'unnamed 0' column cause its just the index of the meta data column
merged_df.drop(columns='Unnamed: 0', inplace=True)

In [11]:
#Check all unique values for each column, this will help us check the nature of
#each variable and if there are more missing values listed as '?' or 'missing'

#Print unique values for each variable
print("Unique values in 'overall':", merged_df['overall'].unique(), "\n")
print("Unique values in 'verified':", merged_df['verified'].unique(), "\n")
print("Unique values in 'reviewTime':", merged_df['reviewTime'].unique(), "\n")
print("Unique values in 'reviewerID':", merged_df['reviewerID'].unique(), "\n")
print("Unique values in 'asin':", merged_df['asin'].unique(), "\n")
print("Unique values in 'style':", merged_df['style'].unique(), "\n")
print("Unique values in 'reviewerName':", merged_df['reviewerName'].unique(), "\n")
print("Unique values in 'reviewText':", merged_df['reviewText'].unique(), "\n")
print("Unique values in 'summary':", merged_df['summary'].unique(), "\n")
print("Unique values in 'unixReviewTime':", merged_df['unixReviewTime'].unique(), "\n")
print("Unique values in 'vote':", merged_df['vote'].unique(), "\n")

Unique values in 'overall': [5 3 4 2 1] 

Unique values in 'verified': [False  True] 

Unique values in 'reviewTime': ['01 23, 2017' '08 3, 2014' '07 9, 2013' ... '06 1, 2002' '10 3, 1999'
 '04 6, 2001'] 

Unique values in 'reviewerID': ['A1YTZAZZ1L32RL' 'A3EHZMD24B5BQ0' 'A38ZMH9LRCZL3X' ... 'A1ZB28DKVCEKAG'
 'A27NFSLWVBJK32' 'ADY836HK6QSYR'] 

Unique values in 'asin': ['0002005263' '0001381733' '0001384198' ... '0060197552' '0060255404'
 '0060224827'] 

Unique values in 'style': ['Kindle Edition' 'Paperback' 'Hardcover' 'Unknown Binding'
 'Mass Market Paperback' 'Board book' 'Library Binding' nan
 'Audible Audiobook' 'Audio CD' 'Audio Cassette' 'Bonded Leather'
 'School & Library Binding' 'Leather Bound' 'Vinyl' 'MP3 CD'
 'Imitation Leather' 'Spiral-bound' 'MP3 Music' 'Amazon Video' 'DVD'
 'VHS Tape' 'Loose Leaf' 'Kindle Edition with Audio/Video' 'Vinyl Bound'
 'Cards' 'Rag Book' 'Digital' 'MP3 CD Library Binding' 'Map' 'Flexibound'
 'DVD Audio' 'Poster' 'Calendar' 'DVD-ROM' 'Plastic 

In [12]:
#Drop reviewerName, reviewerID, and unixReviewTime(we already have a review time var)
merged_df.drop(columns=['reviewerName','reviewerID', 'unixReviewTime'], inplace=True)

In [13]:
#Convert reviewTime to datetime (previously an object type)
merged_df['reviewTime'] = pd.to_datetime(merged_df['reviewTime'], format='%m %d, %Y')

Note: `asin` is a product code, I left in in because maybe we can figure out a way to get more product details and names from this

In [14]:
#Check distribution of scores
print('Distribution of Book overall ratings:')
merged_df['overall'].value_counts(normalize=True)

Distribution of Book overall ratings:


5    0.628216
4    0.203748
3    0.090851
2    0.041702
1    0.035483
Name: overall, dtype: float64

In [15]:
print('Distribution of Book formats')
merged_df['style'].value_counts(normalize=True)

Distribution of Book formats


Kindle Edition                     0.484896
Paperback                          0.227944
Hardcover                          0.187049
Mass Market Paperback              0.066129
Audio CD                           0.010327
Board book                         0.009109
Audible Audiobook                  0.002316
Audio Cassette                     0.001853
Library Binding                    0.001544
Kindle Edition with Audio/Video    0.001424
Spiral-bound                       0.001132
Leather Bound                      0.000738
DVD                                0.000703
Imitation Leather                  0.000635
MP3 CD                             0.000618
Unknown Binding                    0.000480
Amazon Video                       0.000446
Hardcover-spiral                   0.000446
School & Library Binding           0.000377
Map                                0.000257
MP3 Music                          0.000206
Ring-bound                         0.000206
Plastic Comb                    

In [17]:
merged_df.head()

,overall,verified,reviewTime,asin,style,reviewText,summary,vote,Unnamed: 0,title,price
0,5,False,2017-01-23,0002005263,Kindle Edition,Ok reading,Ok,NaN,350,The Sinister Pig,$5.85
1,5,True,2014-08-03,0002005263,Paperback,Good,Five Stars,NaN,350,The Sinister Pig,$5.85
2,5,True,2013-07-09,0001381733,Hardcover,I purchased it for my new great-granddaughter....,New Book for New Grandchild,NaN,88,A Child's Garden of Verses,NaN
3,5,True,2017-12-03,0001384198,Hardcover,A very nice replica of the original classic.,Excellent reproduction of the timeless classic.,NaN,83,The Little Engine That Could,$16.04
4,3,False,2011-10-04,0002005263,Unknown Binding,If you're thinking of buying The Sinister Pig ...,And the hero is . . .,NaN,350,The Sinister Pig,$5.85


In [46]:
merged_df.isna().sum() / len(merged_df) * 100

overall       0.0
verified      0.0
reviewTime    0.0
asin          0.0
style         0.0
reviewText    0.0
summary       0.0
vote          0.0
title         0.0
price         0.0
dtype: float64

In [40]:
#Filling in the null values, 0 for vote (no one voted that review to be helpful), style as unknown, and price as median as per descriptive statistics

merged_df['vote'] = merged_df['vote'].fillna(0)
merged_df['style'] = merged_df['style'].fillna("unknown")
merged_df['price'] = merged_df['price'].fillna(13.39)

In [ ]:
#cleaning up the price column which has this one formatting issue and changing the dtype to int

merged_df['price'] = merged_df['price'].replace(".a-section.a-spacing-mini{margin-bottom:6px!important;margin-top:10px}#actionPanel #availability{font-size:17px!important;font-weight:400!important}#actionPanel #availability p#pa_other_website_text{color:#888!important;margin-top:6px}#actionPanel #merchant-info{font-size:13px;color:#333}#actionPanel #pa_avaliability_message{font-size:12px!important;font-weight:400!important}#actionPanel #pa_avaliability_message a{white-space:normal!important}#actionPanel #pa_avaliability_message span{color:#888!important}#actionPanel #availability-brief #availability+br{display:none}// This is an empty file #actionPanel #bbop-sbbop-container{margin-bottom:0}#actionPanel .buybox-main{padding:0 40px}#actionPanel #priceblock_ourprice_row span.feature img{padding-top:8px!important}#actionPanel #onetimeOption .a-radio-label #priceBadging_feature_div a:not(.a-popover-trigger),#actionPanel #usedOption .a-radio-label #priceBadging_feature_div a:not(.a-popover-trigger){display:none}#actionPanel #pa_feedbackForm_rootmain{text-align:center}.burj #buybox_feature_div #deal #regularBuybox{margin-bottom:14px!important}.burj #hero-quick-promo hr{display:none}#actionPanel #hqp{margin-top:26px}#actionPanel #hqp #amsDetailRight{margin-bottom:18px}#actionPanel #hqp #amsDetailRightWide{margin-bottom:18px}#actionPanel #hqp #hqp-left{margin-bottom:18px}div#hqp-bottom.a-section.burj{margin-bottom:0}.burj #instantOrderUpdate_feature_div", np.NaN)
merged_df['price'] = merged_df['price'].str.replace('$', '').astype(float)

In [42]:
merged_df['price'].describe()

count    61100.000000
mean        21.437621
std         47.667341
min          0.010000
25%          9.800000
50%         13.390000
75%         18.760000
max        973.560000
Name: price, dtype: float64

In [45]:
#Dropping null values for review and summary since they have such low %

merged_df.dropna(inplace=True)

In [76]:
merged_df['vote'].unique()

array([0, '13', '22', '6', '3', '4', '11', '2', '17', '5', '8', '19',
       '48', '7', '10', '26', '120', '25', '9', '42', '50', '16', '21',
       '15', '12', '43', '31', '20', '37', '36', '28', '14', '46', '18',
       '47', '34', '45', '79', '24', '40', '84', '77', '144', '27', '32',
       '96', '254', '148', '23', '892', '104', '67', '29', '38', '143',
       '35', '152', '252', '39', '91', '30', '229', '103', '116', '82',
       '52', '66', '199', '49', '33', '69', '162', '58', '122', '105',
       '107', '95', '190', '61', '202', '175', '53', '55', '59', '123',
       '406', '94', '80', '51', '260', '70', '85', '76', '41', '101',
       '81', '231', '60', '89', '87', '44', '119', '312', '114', '220',
       '727', '65', '292', '93', '88', '54', '100', '108', '606', '92',
       '128', '75', '457', '299', '358', '550', '164', '132', '146',
       '548', '668', '244', '289', '57', '62', '112', '171', '265', '63',
       '291', '139', '73', '161', '201', '106', '388', '68', '213',

In [77]:
#Changing the dtype of verified to 0 and 1, changing the dtype of vote to int
merged_df['verified'] = merged_df['verified'].astype(int)
merged_df['vote'] = merged_df['vote'].replace('6,547', '6547').astype(int)

In [61]:
#creating and adding a new column that has the number of reviews for each book

count_df = pd.DataFrame(merged_df['asin'].value_counts())
count_df = count_df.reset_index()
count_df.columns = ['asin', 'total_review_count']

merged_df = pd.merge(merged_df, count_df, on='asin', how='left')

In [63]:
#adding columns for year, month, date so we can use the reviewtime column for clustering

merged_df['year'] = merged_df['reviewTime'].dt.year
merged_df['day_of_week'] = merged_df['reviewTime'].dt.dayofweek
merged_df['month'] = merged_df['reviewTime'].dt.month
merged_df

,overall,verified,reviewTime,asin,style,reviewText,summary,vote,title,price,total_review_count,year,day_of_week,month
0,5,False,2017-01-23,0002005263,Kindle Edition,Ok reading,Ok,0,The Sinister Pig,5.85,23,2017,0,1
1,5,True,2014-08-03,0002005263,Paperback,Good,Five Stars,0,The Sinister Pig,5.85,23,2014,6,8
2,5,True,2013-07-09,0001381733,Hardcover,I purchased it for my new great-granddaughter....,New Book for New Grandchild,0,A Child's Garden of Verses,13.39,24,2013,1,7
3,5,True,2017-12-03,0001384198,Hardcover,A very nice replica of the original classic.,Excellent reproduction of the timeless classic.,0,The Little Engine That Could,16.04,46,2017,6,12
4,3,False,2011-10-04,0002005263,Unknown Binding,If you're thinking of buying The Sinister Pig ...,And the hero is . . .,0,The Sinister Pig,5.85,23,2011,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61080,5,False,2001-02-02,0060199652,Hardcover,"Barbara Kingsolver's ""Poisonwood Bible"" was a ...","Great characters, good story and an ecology le...",33,Prodigal Summer,8.11,41,2001,4,2
61081,5,True,2013-11-14,0060239492,Hardcover,Have always loved this book. Had it for my ch...,Love this book!,0,Owl at Home (I Can Read Level 2),17.89,8,2013,3,11
61082,4,True,2017-01-12,0060254939,Hardcover,This is a great book.,Four Stars,0,Where the Wild Things Are (Caldecott Collection),13.39,20,2017,3,1
61083,5,True,2014-10-01,0060256729,Hardcover,This is one book of two by Shel Silverstein th...,Love love love this book!,5,The Missing Piece,13.39,12,2014,2,10


In [66]:
merged_df

,overall,verified,reviewTime,asin,style,reviewText,summary,vote,title,price,total_review_count,year,day_of_week,month
0,5,0,2017-01-23,0002005263,Kindle Edition,Ok reading,Ok,0,The Sinister Pig,5.85,23,2017,0,1
1,5,1,2014-08-03,0002005263,Paperback,Good,Five Stars,0,The Sinister Pig,5.85,23,2014,6,8
2,5,1,2013-07-09,0001381733,Hardcover,I purchased it for my new great-granddaughter....,New Book for New Grandchild,0,A Child's Garden of Verses,13.39,24,2013,1,7
3,5,1,2017-12-03,0001384198,Hardcover,A very nice replica of the original classic.,Excellent reproduction of the timeless classic.,0,The Little Engine That Could,16.04,46,2017,6,12
4,3,0,2011-10-04,0002005263,Unknown Binding,If you're thinking of buying The Sinister Pig ...,And the hero is . . .,0,The Sinister Pig,5.85,23,2011,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61080,5,0,2001-02-02,0060199652,Hardcover,"Barbara Kingsolver's ""Poisonwood Bible"" was a ...","Great characters, good story and an ecology le...",33,Prodigal Summer,8.11,41,2001,4,2
61081,5,1,2013-11-14,0060239492,Hardcover,Have always loved this book. Had it for my ch...,Love this book!,0,Owl at Home (I Can Read Level 2),17.89,8,2013,3,11
61082,4,1,2017-01-12,0060254939,Hardcover,This is a great book.,Four Stars,0,Where the Wild Things Are (Caldecott Collection),13.39,20,2017,3,1
61083,5,1,2014-10-01,0060256729,Hardcover,This is one book of two by Shel Silverstein th...,Love love love this book!,5,The Missing Piece,13.39,12,2014,2,10


In [79]:
#pulling out the variables we want

clustering_df = merged_df[['overall', 'verified', 'vote', 'price', 'total_review_count', 'year', 'day_of_week', 'month']]

In [80]:
clustering_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61085 entries, 0 to 61084
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   overall             61085 non-null  int64  
 1   verified            61085 non-null  int64  
 2   vote                61085 non-null  int64  
 3   price               61085 non-null  float64
 4   total_review_count  61085 non-null  int64  
 5   year                61085 non-null  int64  
 6   day_of_week         61085 non-null  int64  
 7   month               61085 non-null  int64  
dtypes: float64(1), int64(7)
memory usage: 4.2 MB


In [81]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
normalized_data = sc.fit_transform(clustering_df)

df_normalized = pd.DataFrame(normalized_data, columns=clustering_df.columns)

df_normalized

,overall,verified,vote,price,total_review_count,year,day_of_week,month
0,0.626606,-1.530896,-0.071172,-0.327031,-0.502064,0.986496,-1.470222,-1.518402
1,0.626606,0.653212,-0.071172,-0.327031,-0.502064,0.254764,1.527125,0.476158
2,0.626606,0.653212,-0.071172,-0.168868,-0.500110,0.010853,-0.970664,0.191221
3,0.626606,0.653212,-0.071172,-0.113281,-0.457114,0.986496,1.527125,1.615906
4,-1.293830,-1.530896,-0.071172,-0.327031,-0.502064,-0.476969,-0.970664,1.046032
...,...,...,...,...,...,...,...,...
61080,0.626606,-1.530896,1.032160,-0.279624,-0.466886,-2.916076,0.528009,-1.233465
61081,0.626606,0.653212,-0.071172,-0.074474,-0.531379,0.010853,0.028452,1.330969
61082,-0.333612,0.653212,-0.071172,-0.168868,-0.507927,0.986496,0.028452,-1.518402
61083,0.626606,0.653212,0.095999,-0.168868,-0.523562,0.254764,-0.471106,1.046032


In [ ]:
#clusteringggg
from scipy.cluster.hierarchy import linkage, fcluster

Z = linkage(df_normalized, 'ward')
num_clusters = 5
clustering_df['cluster_labels'] = fcluster(Z, num_clusters, criterion='maxclust')

In [ ]:
sns.pairplot(clustering_df, hue='cluster_labels')